# Preprocessing the mp3 files and metadata to prepare for the dataloader for the MagnaTagATune


In [2]:
# Importing necessary libraries
# TODO: get rid of the tensorflow warnings

import pandas as pd
import numpy as np
from zipfile import ZipFile 
import os
import librosa
from matplotlib import pyplot as plt
import cv2
import json
pd.options.mode.chained_assignment = None  # default='warn'
from imgaug import augmenters as iaa
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from keras.models import load_model
from keras.callbacks import EarlyStopping
from transformers import AutoTokenizer
import numpy as np
import json
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from collections import OrderedDict

## Loading the zipped files and setting up an audio folder in data/magnatagatune

Downloaded the audio data files from: https://mirg.city.ac.uk/codeapps/the-magnatagatune-dataset
Concatenate the 3 files with `cat mp3.zip.* > mp3_all.zip` in the terminal and unzip them with the code below

In [3]:
'''To unzip the zipped file of mp3 files'''
# # loading the zip file and creating a zip object 
# with ZipFile("/home/mendu/Thesis/data/magnatagatune/mp3_all.zip", 'r') as zObject: 
#     zObject.extractall(path="/home/mendu/Thesis/data/magnatagatune/audio") 

'To unzip the zipped file of mp3 files'

I ran `find audio -type f -name "*.mp3" -exec mv {} audio \;` to get all the .mp3 files into the audio folder and
`find audio -type d -empty -delete` to delete the empty folders

In [4]:
# Checking the no. of audio files
audio_dir = '/home/mendu/Thesis/data/magnatagatune/audio'
audio_list = os.listdir(audio_dir)
audio_list = [file_name.replace('.wav.mp3', '') for file_name in audio_list] # list of audio track names
print('Total number of audio files', len(audio_list))

Total number of audio files 25863


## Filtering the tags dataframe

In [5]:
#Importing the tags csv file
df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/tag_annotations.csv',sep = "\t" ,index_col=[0])
df['mp3_path'] = [i[2:] for i in df['mp3_path']] 

genre: classical, techno, electronic, rock, indian, opera, pop, classic, new age, dance, country, metal

In [7]:
'''genre and synonym_dict for num_class = 8'''
# List of the most frequent genre tags
genre = ['classical', 'clasical', 'classic', 'techno', 'electronic', 'electro', 'electronica', 
 'electric', 'rock', 'indian', 'india', 'opera', 'operatic', 'pop', 'new age', 'dance', 
 'country', 'metal', 'heavy', 'heavy metal']

# Dictionary of genre tags that are synonyms
synonym_dict = {'classical':['classical', 'clasical', 'classic'],
                'electronic': ['electronic', 'electro', 'electronica', 'electric', 'techno', 'dance'],
                'indian': ['indian', 'india'],
                'opera': ['opera', 'operatic'],
                'rock': ['rock', 'metal', 'heavy', 'heavy metal']}

'''genre and synonym_dict for num_class = 10'''
# genre = ['classical', 'clasical', 'classic', 'techno', 'electronic', 'electro', 'electronica', 
#  'electric', 'rock', 'indian', 'india', 'opera', 'operatic', 'pop', 'new age',
#  'country']

# synonym_dict = {'classical':['classical', 'clasical', 'classic'],
#                 'electronic': ['electronic', 'electro', 'electronica', 'electric', 'techno'],
#                 'indian': ['indian', 'india'],
#                 'opera': ['opera', 'operatic'],
#                 'metal': ['metal', 'heavy', 'heavy metal']}


genre.append('mp3_path')

df = df[genre]

In [8]:
# Creating a single column for all the tags
df['tags'] = df.apply(lambda row: (','.join(row.index[row == 1])).split(','), axis=1)

df = df[['mp3_path', 'tags']]

In [9]:
# nested for loop to replace the synonyms
for idx, row in df.iterrows(): #iterates over the rows
    new_tags = []
    for tag in row['tags']: #iterates over each element in the aspect_list list
        for key, values in synonym_dict.items(): #iterates over same_tags dictionary
            if tag in values:
                new_tags.append(key)
                break
        else:
            new_tags.append(tag)
    
    # Making sure the tags are unique in each row        
    new_tags = list(set(new_tags))       
    
    # If multiple tags drop classical if available 
    if len(row['tags'])>1 and 'classical' in row['tags']: #if that are more than one tags and one of it is instrumental drop instrumental
        new_tags.remove('classical')
        
    # updating the tags     
    df.at[idx, 'tags'] = new_tags


# removing empty tags rows
df = df[df['tags'].apply(lambda x: any(x))]

# removing rows with multiple tags
df = df[df['tags'].apply(lambda x: len(x) == 1)].reset_index(drop=True)

# converting the values in the tags column from list to str
df['tags'] = df['tags'].apply(lambda x: x[0])


In [10]:
df.tags.value_counts()

tags
electronic    3279
classical     3191
rock          1751
opera         1259
indian        1234
pop            431
country        354
new age        328
Name: count, dtype: int64

## Generating mel-spectrograms

In [11]:
# Function to convert mp3 files to mel-spectrograms
def mp3_to_melspectrogram(input_folder, output_folder, mp3_name_list):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List to store the names of problematic files
    problematic_files = []

    # Iterate over each file in the input folder
    for filename in mp3_name_list:
        try:
            # Load the audio file
            audio_path = os.path.join(input_folder, filename)
            y, sr = librosa.load(audio_path)

            # Compute the mel-spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

            # Convert to decibel scale
            mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Save the mel-spectrogram as .npy file
            output_path = os.path.join(output_folder, filename.replace('.mp3', '.npy'))
            np.save(output_path, mel_spectrogram_db)

            # print(f"Mel-spectrogram saved: {output_path}")
        
        except Exception as e:
            # Handle the exception (e.g., skip the file, log the filename)
            print(f"Error processing {filename}: {str(e)}")
            problematic_files.append(filename)

    # Return the list of problematic files
    return problematic_files

In [12]:
#generating mel-specs
input_folder = audio_dir
output_folder = '/home/mendu/Thesis/data/magnatagatune/mel-specs-new'
# mp3_to_melspectrogram(input_folder, output_folder, df.mp3_path.tolist())

In [13]:
len(os.listdir('/home/mendu/Thesis/data/magnatagatune/mel-specs-new'))

11826

In [14]:
# removing the .mp3 file causing problems
problematic_file = 'jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3'

df = df[df['mp3_path'] != problematic_file]

In [15]:
#Checking if there is a mel-spec for every row in the df
df_filenames = df.mp3_path.apply(lambda x: x[:-4]).tolist()
mel_list = [x[:-4] for x in os.listdir(output_folder)]

if sorted(df_filenames) == sorted(mel_list):
    print('There is a mel_spectrogram for every row in the df.')
else:
    print('Every row in the df doesnt have a mel-spectrogram.')

There is a mel_spectrogram for every row in the df.


In [16]:
# Elements in list1 but not in list2
only_in_df = [x for x in df_filenames if x not in mel_list]

# Elements in list2 but not in list1
only_in_mel = [x for x in mel_list if x not in df_filenames]

print("Elements in df_filenames but not in mel_list:", len(only_in_df))
print("Elements in mel_list but not in df_filenames:", len(only_in_mel))

Elements in df_filenames but not in mel_list: 0
Elements in mel_list but not in df_filenames: 0


In [17]:
df

,mp3_path,tags
0,american_bach_soloists-j_s__bach_solo_cantatas...,opera
1,american_bach_soloists-j_s__bach_solo_cantatas...,opera
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera
3,american_bach_soloists-j_s__bach_solo_cantatas...,opera
4,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic
...,...,...
11822,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
11823,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
11824,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
11825,jacob_heringman-blame_not_my_lute-57-lost_is_m...,classical


In [19]:
df.to_csv('/home/mendu/Thesis/data/magnatagatune/processed_df.csv', index = False)

In [22]:
new_df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/processed_df.csv')

In [23]:
new_df

,mp3_path,tags
0,american_bach_soloists-j_s__bach_solo_cantatas...,opera
1,american_bach_soloists-j_s__bach_solo_cantatas...,opera
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera
3,american_bach_soloists-j_s__bach_solo_cantatas...,opera
4,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic
...,...,...
11821,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
11822,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
11823,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
11824,jacob_heringman-blame_not_my_lute-57-lost_is_m...,classical
